In [ ]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

from src.utils import get_default_pydantic_model, SideEffect, SideEffectList

import pandas as pd
import torch

import json

import numpy as np

from transformers import BertModel, BertTokenizer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import auc, roc_curve, precision_recall_curve, average_precision_score

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pandas as pd

from itertools import permutations




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

default_model = get_default_pydantic_model("side_effects")
model = BertModel.from_pretrained(os.path.join(paths.MODEL_PATH, "medbert-512")).to(device)
tokenizer = BertTokenizer.from_pretrained(os.path.join(paths.MODEL_PATH, "medbert-512"))

In [ ]:
def get_cosine_similarity(ground_truth:str, prediction:str, model:BertModel, tokenizer:BertTokenizer):
    """
    Calculate if the cosine similarity between the ground truth and the prediction is higher than the threshhold.

    Args:
        ground_truth (str): The ground truth text.
        prediction (str): The predicted text.
        model (BertModel): BertModel object to calculate the cosine similarity.
        tokenizer (BertTokenizer): BertTokenizer object to tokenize the text.

    Returns:
        bool: True if the cosine similarity is higher than the threshhold, False otherwise.
    """
    prediction = prediction.split(",")
    prediction = [", ".join(tup) for tup in list(permutations(prediction))]
    ground_truth = [ground_truth]*len(prediction)

    with torch.no_grad():
        ground_truth = model(**tokenizer(ground_truth, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device))["last_hidden_state"].to("cpu")
        prediction = model(**tokenizer(prediction, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device))["last_hidden_state"].to("cpu")
        ground_truth = ground_truth.mean(dim=1)
        prediction = prediction.mean(dim=1)
    
    similarity = cosine_similarity(ground_truth, prediction)
    max_similarity = np.max(similarity)

    return max_similarity

def preprocess_group(grouped_df: pd.DataFrame)->pd.DataFrame:
    """
    Preprocess a group of rows (grouped by obs index) from the dataframe. The preprocessing consists of:
    - Fusing all the unique side effects per medication with "," (so only one row per medication is left)
    - Splitting the medications by "/" if necessary

    Args:
        grouped_df (pd.DataFrame): A group of rows from the dataframe.

    Returns:
        pd.DataFrame: A dataframe with the group preprocessed.
    """

    df = []
    
    # group by medication name
    for med, group in grouped_df.groupby("medication"):
       # Fuse all the unique side effects with ","
        side_effects = group["side_effect"].unique()
        side_effects = ", ".join(side_effects)
        text = group["text"].iloc[0]
        original_text = group["original_text"].iloc[0]
        index = group["index"].iloc[0]

        new_row = {"medication": med, "side_effect": side_effects, "text": text, "original_text": original_text, "index": index, "successful": group["successful"].iloc[0]}
        # Append the new row to the dataframe
        df.append(new_row)

    return pd.DataFrame(df)

def prepare_results(path: str)->pd.DataFrame:
    """ 
    Prepare the results from the model to be compared with the labels. The preprocessing consists of:
    - Fixing the model answers, replacing them with the default model answer if they are invalid (in regards to JSON structure).
    - Grouping the results by index and preprocessing the group. (See preprocess_group() function for more details.)
    - Lowercasing and converting everything to string.

    Args:
        path (str): Path to the results file.
        
    Returns:
        pd.DataFrame: A dataframe with the results preprocessed.
    """
    results = torch.load(path)
    df = pd.DataFrame(results)

    df["successful"] = True

    # Fix model_answers
    for idx, row in df.iterrows():
        try:
            # Does model answer have key "side_effects"?
            answer = json.loads(row["model_answers"])

            # Is answer a valid SideEffectList?
            SideEffectList(**answer)

            # Is the element an empty list?
            if len(answer["side_effects"]) == 0:
                raise ValueError("Empty list")
            
            # Are all the elements valid SideEffect?
            for med in answer["side_effects"]:
                SideEffect(**med)
            
            df.at[idx, "model_answers"] = json.dumps(answer)

            
        except:
            df.at[idx, "model_answers"] = default_model.model_dump_json()
            df.at[idx, "successful"] = False
    
    dfs = []
    for idx, row in df.iterrows():
        row = row.to_dict()
        answer = json.loads(row["model_answers"])
        medications = answer["side_effects"]
        for med in medications:
            med["text"] = row.get("text", "")
            med["original_text"] = row.get("original_text", "")
            med["index"] = row.get("index", "")
            med["successful"] = row.get("successful", "")
            dfs.append((med))

    res = pd.DataFrame(dfs)

    res = res.sort_values(by="index")

    # Group by index and preprocess
    _dfs = []

    for index, group in res.groupby("index"):
        _dfs.append(preprocess_group(group))

    res = pd.concat(_dfs)

    # Convert everything to string and lowercase
    res = res.map(lambda x: str(x).lower())

    return res

def prepare_labels(path: str)->pd.DataFrame:
    """
    Prepare the labels to be compared with the results. The preprocessing consists of:
    - Lowercasing and converting everything to string.

    Args:
        path (str): Path to the labels file.

    Returns:

        pd.DataFrame: A dataframe with the labels preprocessed.
    """
    labels = pd.read_excel(path) 

    # Convert everything to string and lowercase
    labels = labels.map(lambda x: str(x).lower())

    return labels

def calculate_precision_recall_f1(ground_truth:list[str], predicted:list[str])->tuple[float, float, float]:
    """
    Calculate precision and recall from two lists of strings, allowing for partial string matches.
    A string does not have to be exactly the same to be considered a match, it is enough if one string is a substring of the other.
    This function is used to evaluate the medication names.

    Args:
        ground_truth (list[str]): List of ground truth medication names
        predicted (list[str]): List of predicted medication names

    Returns:
        tuple[float, float, float]: A tuple with the precision, recall and f1 score.
    """
    # Convert lists to sets for easier comparison
    ground_truth_set = set(ground_truth)
    predicted_set = set(predicted)

    # Initialize counters
    true_positives = 0
    false_positives = 0

    # Calculate True Positives and False Positives
    for pred_med in predicted_set:
        found_match = False
        for truth_med in ground_truth_set:
            if pred_med in truth_med or truth_med in pred_med:
                true_positives += 1
                found_match = True
                break
        if not found_match:
            false_positives += 1

    # Calculate False Negatives
    false_negatives = len(ground_truth_set) - true_positives

    # Calculate Precision
    if true_positives + false_positives == 0:
        precision = 0  # Handle division by zero
    else:
        precision = true_positives / (true_positives + false_positives)

    # Calculate Recall
    if true_positives + false_negatives == 0:
        recall = 0  # Handle division by zero
    else:
        recall = true_positives / (true_positives + false_negatives)

    # Calculate F1 Score
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1

def string_match(ground_truth: list[dict], predicted: list[dict], model: BertModel, tokenizer: BertTokenizer, threshold: float = 0.9) -> tuple[float, float, float]:
    """
    Calculate if the strings match (based on cosine similarity) from two lists of dictionaries.
    """
    tp = 0  # True Positives

    # Copy the ground truth and predicted lists to avoid modifying the original lists
    remaining_gt = ground_truth.copy()
    matched_gt_indices = set()

    for pred in predicted:
        for gt_index, gt in enumerate(remaining_gt):
            if gt_index in matched_gt_indices:
                # Skip already matched ground truth items
                continue
            
            if pred["medication"] in gt["medication"] or gt["medication"] in pred["medication"]:
                if get_cosine_similarity(pred["side_effect"], gt["side_effect"], model, tokenizer) > threshold:
                    tp += 1
                    matched_gt_indices.add(gt_index)
                    break


    precision = tp/len(predicted)
    recall = tp/len(ground_truth)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1


def create_metrics_df(labels_path:str, results_path:str, *args, **kwargs)->pd.DataFrame:
    """
    Create a dataframe with the metrics for each index. The metrics are:
    - Precision, Recall and F1 score for medication names
    - Precision, Recall and F1 score for side effects

    Args:
        labels_path (str): Path to the labels file.
        results_path (str): Path to the results file.

    Returns:
        pd.DataFrame: A dataframe with the metrics for each index.
    """
    
    labels = prepare_labels(labels_path)
    res = prepare_results(results_path)
    model = kwargs.get("model")
    tokenizer = kwargs.get("tokenizer")
    threshold = kwargs.get("threshold", 0.9)

    dfs = []
    for index, group in res.groupby("index"):
        text = group["text"].iloc[0]
        original_text = group["original_text"].iloc[0]
        index = group["index"].iloc[0]
        preds_med = group["medication"].to_list()
        preds_se = group["side_effect"].to_list()
        labels_med = labels[labels["index"] == index]["medication"].to_list()
        labels_se = labels[labels["index"] == index]["side_effect"].to_list()

        # Side effects
        ground_truth = labels[labels["index"] == index].to_dict(orient="records")
        predicted = group.to_dict(orient="records")
        assert model is not None and tokenizer is not None, "Model and tokenizer must be provided as keyword arguments for side effects evaluation."
        string_match(ground_truth, predicted, model, tokenizer, threshold=threshold)
        side_effects_precision, side_effects_recall, side_effects_f1 = string_match(ground_truth, predicted, model, tokenizer, threshold=threshold)

        # Medication
        ground_truth = labels[labels["index"] == index]["medication"].to_list()
        predicted = group["medication"].to_list()
        medication_precision, medication_recall, medication_f1 = calculate_precision_recall_f1(ground_truth, predicted)

        
        if group["successful"].iloc[0] == "false":
            successful = False
        else:
            successful = True

        dfs.append({"text": text, 
                    "original_text": original_text, 
                    "index": index, 
                    "preds_med": preds_med,
                    "labels_med": labels_med,
                    "preds_se": preds_se,
                    "labels_se": labels_se,
                    "successful": successful,
                    "side_effects_precision": side_effects_precision, "side_effects_recall": side_effects_recall, "side_effects_f1": side_effects_f1, "medication_precision": medication_precision, "medication_recall": medication_recall, "medication_f1": medication_f1})

    return pd.DataFrame(dfs)

def create_agg_metrics_df(labels_path:str, results_path:str, *args, **kwargs)->pd.DataFrame:
    """
    Create a dataframe with the aggregated metrics for the whole dataset. The metrics are:
    - Precision, Recall and F1 score for medication names
    - Precision, Recall and F1 score for side effects

    Args:
        labels_path (str): Path to the labels file.
        results_path (str): Path to the results file.
        scorer (BERTScorer): BERTScorer object to score the similarity between the side effects.

    Returns:
        pd.DataFrame: A dataframe with the aggregated metrics for the whole dataset.
    """
    metrics = create_metrics_df(labels_path, results_path, *args, **kwargs)

    # Calculate the average of the metrics
    agg_metrics = metrics.agg({"side_effects_precision": "mean", "side_effects_recall": "mean", "side_effects_f1": "mean", "medication_precision": "mean", "medication_recall": "mean", "medication_f1": "mean", "successful": "sum"})

    return agg_metrics

In [ ]:
def summarize(paths:list[str], labels:str, *args, **kwargs)->pd.DataFrame:
    """
    Summarize the results from the model. The summary consists of:
    - Precision, Recall and F1 score for medication names
    - Precision, Recall and F1 score for side effects
    - Aggregated metrics for the whole dataset

    Args:
        paths (list): List of paths to the results files.
        labels (str): Path to the labels file.

    Returns:
        pd.DataFrame: A dataframe with the summary of the results.
    """
    dfs = []
    for path in paths:
        metrics = create_agg_metrics_df(labels, path, *args, **kwargs)
        path = str(path)
        if path.endswith("rag.pt"):
            metrics["approach"] = "S2A-1"
        elif path.endswith("s2a.pt"):
            metrics["approach"] = "S2A-2"
        else:
            metrics["approach"] = "Base"
        
        if "few_shot_vanilla" in path:
            metrics["strategy"] = "Few-Shot Base"
        elif "few_shot_instruction" in path:
            metrics["strategy"] = "Few-Shot Instruction"
        elif "zero_shot_vanilla" in path:
            metrics["strategy"] = "Zero-Shot Base"
        elif "zero_shot_instruction" in path:
            metrics["strategy"] = "Zero-Shot Instruction"
        else:
            ValueError("Unknown strategy")
        dfs.append(metrics)
    return pd.DataFrame(dfs)

# 13B

## Base

In [ ]:
# Zero Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_vanilla.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Zero Shot instruction
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_instruction.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
# Few Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_vanilla_10_examples.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

## RAG

In [ ]:
# Zero Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_vanilla_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Zero Shot instruction
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_instruction_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Few Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_vanilla_10_examples_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

## S2A

In [ ]:
# Zero Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_vanilla_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Zero Shot instruction
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_instruction_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Few Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_vanilla_10_examples_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
# Few Shot Instruction
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

## Summary

In [ ]:
def latex_metrics(metrics:pd.DataFrame)->str:
    """
    Convert the metrics dataframe to a latex table.
    """
    dfs = []
    for strat, group in metrics.groupby("strategy"):
        cols = [col for col in metrics.columns if col not in ["strategy", "approach"]]
        df_base = group[group["approach"] == "Base"][cols].reset_index(drop=True)
        df_base.columns = [f"{col}_base" for col in cols]
        df_rag = group[group["approach"] == "S2A-1"][cols].reset_index(drop=True)
        df_rag.columns = [f"{col}_rag" for col in cols]
        df_s2a = group[group["approach"] == "S2A-2"][cols].reset_index(drop=True)
        df_s2a.columns = [f"{col}_s2a" for col in cols]
        df = pd.concat([df_base, df_rag, df_s2a], axis=1)
        df["strategy"] = strat
        dfs.append(df)

    return pd.concat(dfs, axis=0)

def p_r_f_data(results: pd.DataFrame):
    thesis13b = latex_metrics(results)
    thesis13b_precision = thesis13b[["strategy"] + [col for col in thesis13b.columns if "precision" in col]]
    thesis13b_recall = thesis13b[["strategy"] + [col for col in thesis13b.columns if "recall" in col]]
    thesis13b_f1 = thesis13b[["strategy"] + [col for col in thesis13b.columns if "f1" in col]]
    return thesis13b_precision, thesis13b_recall, thesis13b_f1
    

In [ ]:
file_paths = [paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_vanilla.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_instruction.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_vanilla_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_vanilla_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_instruction_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_vanilla_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_vanilla_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_zero_shot_instruction_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_vanilla_10_examples_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples_s2a.pt",]

In [ ]:
results = summarize(file_paths, paths.RESULTS_PATH/"side-effects/labels.xlsx", model=model, tokenizer=tokenizer, threshold = 0.78)
results = results.round(2)
# results.to_csv(paths.RESULTS_PATH/"side-effects/summary13b.csv", index=False)

In [ ]:
results

In [ ]:
thesis13b_precision,thesis13b_recall, thesis13b_f1 =  p_r_f_data(results)

In [ ]:
# thesis13b = latex_metrics(results)
# thesis13b_precision = thesis13b[["strategy"] + [col for col in thesis13b.columns if "precision" in col]]
# thesis13b_recall = thesis13b[["strategy"] + [col for col in thesis13b.columns if "recall" in col]]
# thesis13b_f1 = thesis13b[["strategy"] + [col for col in thesis13b.columns if "f1" in col]]

# thesis13b_precision.to_csv(paths.RESULTS_PATH/"side-effects/thesis13b_precision.csv", index=False)
# thesis13b_recall.to_csv(paths.RESULTS_PATH/"side-effects/thesis13b_recall.csv", index=False)
# thesis13b_f1.to_csv(paths.RESULTS_PATH/"side-effects/thesis13b_f1.csv", index=False)

- Model seems to have difficulties if there is no medication mentioned with side effects. The tuned model does not have this problem I think, show example
- also split medications that contain "/" into two lists
- concatenate the side effects for the same medications with ","

# 7B

## Base

In [ ]:
# Zero Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_vanilla.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Zero Shot instruction
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_instruction.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Few Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_vanilla_10_examples.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_instruction_10_examples.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

## RAG

In [ ]:
# Zero Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_vanilla_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Zero Shot instruction
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_instruction_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Few Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_vanilla_10_examples_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_instruction_10_examples_rag.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

## S2A

In [ ]:
# Zero Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_vanilla_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Zero Shot instruction
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_instruction_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)


In [ ]:
# Few Shot Vanilla
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_vanilla_10_examples_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
create_agg_metrics_df(paths.RESULTS_PATH/"side-effects/labels.xlsx", paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_instruction_10_examples_s2a.pt", model=model, tokenizer=tokenizer, threshold = 0.5)

In [ ]:
file_paths7b = [paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_vanilla.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_instruction.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_vanilla_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_instruction_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_vanilla_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_instruction_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_vanilla_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_instruction_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_vanilla_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_zero_shot_instruction_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_vanilla_10_examples_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-7b_4bit_few_shot_instruction_10_examples_s2a.pt",]

In [ ]:
results7b = summarize(file_paths7b, paths.RESULTS_PATH/"side-effects/labels.xlsx", model=model, tokenizer=tokenizer, threshold = 0.9)
results7b = results7b.round(2)
# results7b.to_csv(paths.RESULTS_PATH/"side-effects/summary7b.csv", index=False)
# results7b.to_csv(paths.THESIS_PATH/"se_summary7b.csv", index=False)

In [ ]:
results7b

In [ ]:
thesis7b = latex_metrics(results7b)
thesis7b_precision = thesis7b[["strategy"] + [col for col in thesis7b.columns if "precision" in col]]
thesis7b_recall = thesis7b[["strategy"] + [col for col in thesis7b.columns if "recall" in col]]
thesis7b_f1 = thesis7b[["strategy"] + [col for col in thesis7b.columns if "f1" in col]]

# thesis7b_precision.to_csv(paths.RESULTS_PATH/"side-effects/thesis7b_precision.csv", index=False)
# thesis7b_recall.to_csv(paths.RESULTS_PATH/"side-effects/thesis7b_recall.csv", index=False)
# thesis7b_f1.to_csv(paths.RESULTS_PATH/"side-effects/thesis7b_f1.csv", index=False)

thesis7b_precision.to_csv(paths.THESIS_PATH/"se_thesis7b_precision.csv", index=False)
thesis7b_recall.to_csv(paths.THESIS_PATH/"se_thesis7b_recall.csv", index=False)
thesis7b_f1.to_csv(paths.THESIS_PATH/"se_thesis7b_f1.csv", index=False)

# 13B Lora

In [ ]:
file_paths13b_lora_1024 = [
                paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_zero_shot_vanilla.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_zero_shot_instruction.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_vanilla_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_instruction_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_zero_shot_vanilla_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_zero_shot_instruction_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_vanilla_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_instruction_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_zero_shot_vanilla_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_zero_shot_instruction_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_vanilla_10_examples_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_instruction_10_examples_s2a.pt",]

In [ ]:
results13b_lora_1024 = summarize(file_paths13b_lora_1024, paths.RESULTS_PATH/"side-effects/labels.xlsx", model=model, tokenizer=tokenizer, threshold = 0.78)

In [ ]:
results13b_lora_1024 = results13b_lora_1024.round(2)
results13b_lora_1024

In [ ]:
thesis13b_1024_precision,thesis13b_1024_recall, thesis13b_1024_f1 =  p_r_f_data(results13b_lora_1024)

In [ ]:
file_paths13b_lora_512 = [
                paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_zero_shot_vanilla.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_zero_shot_instruction.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_vanilla_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_instruction_10_examples.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_zero_shot_vanilla_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_zero_shot_instruction_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_vanilla_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_instruction_10_examples_rag.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_zero_shot_vanilla_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_zero_shot_instruction_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_vanilla_10_examples_s2a.pt",
              paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_instruction_10_examples_s2a.pt",]

In [ ]:
results13b_lora_512 = summarize(file_paths13b_lora_512, paths.RESULTS_PATH/"side-effects/labels.xlsx", model=model, tokenizer=tokenizer, threshold = 0.78)

In [ ]:
results13b_lora_512 = results13b_lora_512.round(2)
results13b_lora_512

In [ ]:
thesis13b_512_precision,thesis13b_512_recall, thesis13b_512_f1 =  p_r_f_data(results13b_lora_512)

# Thesis
Concat results of 13B models

In [ ]:
overall_13b_precision = pd.concat([thesis13b_precision, thesis13b_512_precision, thesis13b_1024_precision])
overall_13b_precision["model"] = ["base"] * len(thesis13b_precision) + ["512"] * len(thesis13b_precision) + ["1024"] * len(thesis13b_precision)
overall_13b_recall = pd.concat([thesis13b_recall, thesis13b_512_recall, thesis13b_1024_recall])
overall_13b_recall["model"] = ["base"] * len(thesis13b_recall) + ["512"] * len(thesis13b_recall) + ["1024"] * len(thesis13b_recall)
overall_13b_f1 = pd.concat([thesis13b_f1, thesis13b_512_f1, thesis13b_1024_f1])
overall_13b_f1["model"] = ["base"] * len(thesis13b_f1) + ["512"] * len(thesis13b_f1) + ["1024"] * len(thesis13b_f1)

In [ ]:
overall_13b_f1[["instruction" in strat.lower() for strat in overall_13b_f1["strategy"]]].to_csv(paths.THESIS_PATH/"se_lora_f1.csv")
overall_13b_precision[["instruction" in strat.lower() for strat in overall_13b_precision["strategy"]]].to_csv(paths.THESIS_PATH/"se_lora_precision.csv")
overall_13b_recall[["instruction" in strat.lower() for strat in overall_13b_recall["strategy"]]].to_csv(paths.THESIS_PATH/"se_lora_recall.csv")

In [ ]:
overall_13b_f1[["instruction" in strat.lower() for strat in overall_13b_f1["strategy"]]]

# Threshold
Will compare effects of threshold for few shot instruction s2a on results

In [ ]:
df = pd.read_excel(paths.RESULTS_PATH/"side-effects/similarities.xlsx")
df["cosine_sim"] = df.apply(lambda x: get_cosine_similarity(x["gt"], x["pred"], model, tokenizer), axis=1)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
# Calculate best threshold
fpr, tpr, thresholds = roc_curve(df["Similar"], df["cosine_sim"])

# Calculate using the Youden's J statistic; sensitivity + specificity - 1, sensitivity = tpr, specificity = 1 - fpr
optimal_idx_y = np.argmax(tpr - fpr)
optimal_threshold_y = thresholds[optimal_idx_y]

# Calculate using shortest distance to top left
dist = np.sqrt(fpr**2 + (1-tpr)**2)
optimal_idx_d = np.argmin(dist)
optimal_threshold_d = thresholds[optimal_idx_d]

# Calculate by not exceeding 0.1 for fpr
optimal_idx_f = np.abs(fpr - 0.1).argmin()
optimal_threshold_f = thresholds[optimal_idx_f]

In [ ]:
def plot_roc_curve(y_true, y_score, title:str = "ROC Curve", save_dir:str=None):
    """
    Plot the ROC curve.

    Args:
        y_true (np.ndarray): True labels.
        y_score (np.ndarray): Predicted scores.
        title (str): Title of the plot.
        save_dir (str): Path to save the plot.
    """
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(10, 6))
    
    # Use viridis palette for ROC curve - for example, at the 0.75 mark of the palette
    roc_color = cm.viridis(0.75)
    plt.plot(fpr, tpr, color=roc_color, lw=3, label='ROC curve (area = %0.2f)' % roc_auc)
    
    # Use viridis palette for the diagonal line - for example, at the 0.25 mark of the palette
    diagonal_color = cm.viridis(0.25)
    plt.plot([0, 1], [0, 1], color=diagonal_color, lw=3, linestyle='--')

    # Optimal threshold
    plt.axvline(x=fpr[optimal_idx_d], color="blue", linestyle="--", label="Balanced Threshold")
    plt.axvline(x=fpr[optimal_idx_f], color="green", linestyle="--", label="FPR < 0.1 Threshold")
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=20, fontweight='bold', labelpad=10)
    plt.ylabel('True Positive Rate', fontsize=20, fontweight='bold', labelpad=15)
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)

    if save_dir:
        plt.savefig(save_dir)
    plt.show()

def plot_prc_curve(y_true, y_score, title:str = "PRC Curve", save_dir:str=None):
    """
    Plot the Precision-Recall curve.

    Args:
        y_true (np.ndarray): True labels.
        y_score (np.ndarray): Predicted scores.
        title (str): Title of the plot.
        save_dir (str): Path to save the plot.
    """
    precision, recall, thresholds = precision_recall_curve(y_true, y_score)
    prc_auc = average_precision_score(y_true, y_score)

    plt.figure(figsize=(10, 6))
    
    # Use viridis palette for PRC curve - for example, at the 0.75 mark of the palette
    prc_color = cm.viridis(0.5)
    plt.plot(recall, precision, color=prc_color, label='PRC curve (area = %0.2f)' % prc_auc, linewidth=3)

    # Optimal threshold
    closest_threshold_d = np.argmin(np.abs(thresholds - optimal_threshold_d))
    closest_threshold_f = np.argmin(np.abs(thresholds - optimal_threshold_f))
    plt.axvline(x=recall[closest_threshold_d], color="blue", linestyle="--", label="Balanced Threshold")
    plt.axvline(x=recall[closest_threshold_f], color="green", linestyle="--", label="FPR < 0.1 Threshold")
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall', fontsize=20, fontweight='bold', labelpad=10)
    plt.ylabel('Precision', fontsize=20, fontweight='bold', labelpad=15)
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    
    if save_dir:
        plt.savefig(save_dir)
    plt.show()

In [ ]:
plot_roc_curve(df["Similar"], df["cosine_sim"], title="", save_dir=paths.THESIS_PATH/"se_roc.png")

In [ ]:
plot_prc_curve(df["Similar"], df["cosine_sim"], title="", save_dir=paths.THESIS_PATH/"se_prc.png")

In [ ]:
optimal_threshold_y, optimal_threshold_d, optimal_threshold_f

In [ ]:
threshold_files = [
    paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples_s2a.pt",
    paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_instruction_10_examples_s2a.pt",
    paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_instruction_10_examples_s2a.pt",
                   ]

In [ ]:
thresholds = np.linspace(0, 1, 11)
dfs = []
for threshold in thresholds:
    _df = summarize(threshold_files, paths.RESULTS_PATH/"side-effects/labels.xlsx", model=model, tokenizer=tokenizer, threshold = threshold)
    _df["model"] = ["Llama2-MedTuned-13B", "Llama2-MedTuned-13B-lora-512", "Llama2-MedTuned-13B-lora-1024"]
    _df["threshold"] = threshold
    dfs.append(_df)

In [ ]:
df_thresholds = pd.concat(dfs, axis=0)
df_thresholds.drop(columns=["successful", "medication_precision", "medication_recall", "medication_f1",	"successful","approach", "strategy"], inplace=True)

In [ ]:
def plot_line_plot_group(x:pd.Series, y:pd.Series, groups:pd.Series, title:str = "Line Plot",
                             xlabel:str = "Threshold", ylabel:str = "Recall", save_dir:str=None):
    """
    Plot a line plot with different color for group.

    Args:
        x (pd.Series): x values.
        y (pd.Series): y values.
        groups (pd.Series): group values.
        title (str): Title of the plot.
        color_palette (np.ndarray): Color palette to use.
    """

    assert len(x) == len(y) == len(groups), "Length of x, y and group must be the same."

    unique_groups = groups.unique()

    data = pd.DataFrame({"x": x, "y": y, "groups": groups})

    plt.figure(figsize=(10, 6))

    sns.lineplot(data=data, x="x", y="y", hue="groups", palette="viridis")

    plt.title(title)
    plt.xlabel(xlabel, fontsize=20, fontweight='bold', labelpad=10)
    plt.ylabel(ylabel, fontsize=20, fontweight='bold', labelpad=15)

    # Optimal threshold
    plt.axvline(x=optimal_threshold_d, color="blue", linestyle="--", label="Balanced Threshold")
    plt.axvline(x=optimal_threshold_f, color="green", linestyle="--", label="FPR < 0.1 Threshold")

    plt.legend()
    plt.grid(True)
    if save_dir:
        plt.savefig(save_dir)
    plt.show()

In [ ]:
plot_line_plot_group(df_thresholds["threshold"], df_thresholds["side_effects_recall"], df_thresholds["model"], title="", ylabel="Recall", xlabel="Threshold", save_dir = paths.THESIS_PATH/"se_threshold_recall.png")

In [ ]:
plot_line_plot_group(df_thresholds["threshold"], df_thresholds["side_effects_precision"], df_thresholds["model"], title="", ylabel="Precision", xlabel="Threshold", save_dir = paths.THESIS_PATH/"se_threshold_precision.png")

In [ ]:
plot_line_plot_group(df_thresholds["threshold"], df_thresholds["side_effects_f1"], df_thresholds["model"], title="", ylabel="F1", xlabel="Threshold", save_dir = paths.THESIS_PATH/"se_threshold_f1.png")

In [ ]:
def plot_precision_recall_curve(df, title="Precision-Recall Curve", xlabel="Recall", ylabel="Precision", save_dir=None):
    """
    Plot the Precision-Recall curve for different models.

    Args:
        df (pd.DataFrame): DataFrame containing precision, recall, and model name.
        title (str): Title of the plot.
        xlabel (str): X-axis label.
        ylabel (str): Y-axis label.
        save_dir (str, optional): Directory path to save the plot. Defaults to None.
    """

    plt.figure(figsize=(10, 6))

    # Use seaborn lineplot to draw precision-recall curves for each model
    sns.lineplot(data=df, x="side_effects_recall", y="side_effects_precision", hue="model", palette="viridis", style="model", markers=True, dashes=False)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)

    if save_dir:
        plt.savefig(save_dir, bbox_inches='tight')
    plt.show()

In [ ]:
plot_precision_recall_curve(df_thresholds)

# Examples

In [ ]:
res_1024 = prepare_results(paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-1024-lora-merged_4bit_few_shot_instruction_10_examples_s2a.pt")
res_512 = prepare_results(paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b-512-lora-merged_4bit_few_shot_instruction_10_examples_s2a.pt")
res = prepare_results(paths.RESULTS_PATH/"side-effects/side-effects_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction_10_examples_s2a.pt")
labels = prepare_labels(paths.RESULTS_PATH/"side-effects/labels.xlsx")

In [ ]:
for index, data in res_1024.groupby("index"):
    if len(res_512[res_512["index"] == index]) != len(data):
        print("labels")
        display(labels[labels["index"] == index])
        print("1024")
        display(data)
        print("res")
        display(res[res["index"] == index])
        print("512")
        display(res_512[res_512["index"] == index])
        print("-" * 100)

Performance difference:
- index 17: 1024 model understand better, when no medication is given like "unerwünschte nebenwirkung ona arzneimittel oder droge", to just put "unknown", meanwhile the other 2 repeated the examples from few-shot
- index 22 another example: "indent labor: crp <0.5 mg/ml, bsr 8 mm/h, nw leukozyten + thrombozyten", also unknown

Good examples:
- index 3
- index 13

Bad examples:
- index: 55; "indent st.n. symptomatischer therapie mit fampyra (insgesamt 2 therapieversuche, gestoppt bei wirkungslosogkeit und i.r. der kardialen vorbefunden), therapieversuch mit modasomil/remeron (keine besserung), venlafaxin (bei nebenwirkungen gestoppt), sirdalud
indent rebif 2002, abgesetzt 6 monate später wegen unverträglichkeit (grippale nebenwirkungen)"; only venlafaxin and rebif have se (model extracts venlafaxin with unknown, misses rebif and puts all the other medications like fampyra, modasomil, remeron, sirdalud

## Good

In [ ]:
print(res_1024[res_1024["index"] == "3"][res_1024.columns[:-4]].to_dict(orient = "records"))

In [ ]:
print(res_1024[res_1024["index"] == "13"][res_1024.columns[:-4]].to_dict(orient = "records"))

## Bad

In [ ]:
print(res_1024[res_1024["index"] == "22"][res_1024.columns[:-4]].to_dict(orient = "records"))
print(res_512[res_512["index"] == "22"][res_512.columns[:-4]].to_dict(orient = "records"))

In [ ]:
print(res_1024[res_1024["index"] == "55"][res_1024.columns[:-4]].to_dict(orient = "records"))